In [82]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization


In [83]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset ='/home/turning/Desktop/CL_project/CL_PROJECT_CODE/Kerasmodel/aclImdb'

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

['README', 'train']

In [84]:
train_dir = '/home/turning/Desktop/CL_project/CL_PROJECT_CODE/Kerasmodel/aclImdb/train/'
os.listdir(train_dir)

['pos']

In [85]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/home/turning/Desktop/CL_project/CL_PROJECT_CODE/Kerasmodel/aclImdb/train/unsup'

In [87]:
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Found 6 files belonging to 1 classes.
Using 5 files for training.
Found 6 files belonging to 1 classes.
Using 1 files for validation.


In [88]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [89]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(10000, 5)

In [90]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[ 0.03471095, -0.03843842, -0.03425549, -0.02572281,  0.00769442],
       [ 0.00233169, -0.02095674,  0.03440174,  0.03112933, -0.0417734 ],
       [ 0.03936919, -0.04475255,  0.02868069, -0.03221764, -0.04514248]],
      dtype=float32)

In [91]:
result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape

TensorShape([2, 3, 5])

In [102]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 1000000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [103]:
embedding_dim=20

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [104]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [106]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [107]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
1/1 [==============================] - 1s 842ms/step - loss: 0.6929 - accuracy: 1.0000 - val_loss: 0.6886 - val_accuracy: 1.0000
Epoch 2/15
1/1 [==============================] - 0s 133ms/step - loss: 0.6891 - accuracy: 1.0000 - val_loss: 0.6862 - val_accuracy: 1.0000
Epoch 3/15
1/1 [==============================] - 0s 178ms/step - loss: 0.6857 - accuracy: 1.0000 - val_loss: 0.6837 - val_accuracy: 1.0000
Epoch 4/15
1/1 [==============================] - 0s 148ms/step - loss: 0.6822 - accuracy: 1.0000 - val_loss: 0.6812 - val_accuracy: 1.0000
Epoch 5/15
1/1 [==============================] - 0s 133ms/step - loss: 0.6790 - accuracy: 1.0000 - val_loss: 0.6786 - val_accuracy: 1.0000
Epoch 6/15
1/1 [==============================] - 0s 127ms/step - loss: 0.6755 - accuracy: 1.0000 - val_loss: 0.6760 - val_accuracy: 1.0000
Epoch 7/15
1/1 [==============================] - 0s 130ms/step - loss: 0.6720 - accuracy: 1.0000 - val_loss: 0.6734 - val_accuracy: 1.0000
Epoch 8/15
1/1 [====

In [108]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_5 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 100, 20)           20000000  
                                                                 
 global_average_pooling1d_6   (None, 20)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_12 (Dense)            (None, 16)                336       
                                                                 
 dense_13 (Dense)            (None, 1)                 17        
                                                                 
Total params: 20,000,353
Trainable params: 20,000,353


In [98]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 10542), started 5:22:45 ago. (Use '!kill 10542' to kill it.)

In [99]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [100]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [101]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass